In [241]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
from subprocess import check_output
print(check_output(["ls", "../Final_Project/Data"]).decode("utf8"))
# Any results you write to the current directory are saved as output.

from ast import literal_eval
import ast

credits.csv
keywords.csv
links.csv
links_small.csv
movies_Merged_Alldata.csv
movies_Merged_Alldata.csv.gz
movies_metadata.csv
ratings.csv
ratings_small.csv



In [242]:
PATH = "../Final_Project/Data/"

In [243]:
df_credits = pd.read_csv(PATH+"credits.csv")
print("credits: \n ", df_credits.columns)
df_keywords = pd.read_csv(PATH+"keywords.csv")
print("\n keywords: \n", df_keywords.columns)

df_links = pd.read_csv(PATH+"links.csv")
print("\n links: \n", df_links.columns)

credits: 
  Index(['cast', 'crew', 'id'], dtype='object')

 keywords: 
 Index(['id', 'keywords'], dtype='object')

 links: 
 Index(['movieId', 'imdbId', 'tmdbId'], dtype='object')


In [244]:
df = pd.read_csv(PATH+"movies_metadata.csv")
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


# Some columns types are read badly : fix
- ID column should be numeric.
- we can see this using dtype (merge/join also would fail).
- it has no nulls though.
- debug by printing non numeric rows in it:
      - Finally: drop these (few) bad rows

In [245]:
df[["id","imdb_id"]].dtypes

id         object
imdb_id    object
dtype: object

In [246]:
df.isnull().sum()


adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [247]:
# Only 3 bad rows!
print (df[pd.to_numeric(df['id'], errors='coerce').isnull()])

                                                   adult  \
19730                                 - Written by Ørnås   
29503   Rune Balot goes to a casino connected to the ...   
35587   Avalanche Sharks tells the story of a bikini ...   

      belongs_to_collection                            budget  \
19730              0.065736  /ff9qCepilowshEtG2GYWwzt2bs4.jpg   
29503              1.931659  /zV8bHuSL6WXoD6FWogP9j4x80bL.jpg   
35587              2.185485  /zaSf5OG7V8X8gqFvly88zDdRm46.jpg   

                                                  genres  \
19730  [{'name': 'Carousel Productions', 'id': 11176}...   
29503  [{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...   
35587  [{'name': 'Odyssey Media', 'id': 17161}, {'nam...   

                                                homepage          id imdb_id  \
19730  [{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...  1997-08-20       0   
29503  [{'iso_3166_1': 'US', 'name': 'United States o...  2012-09-29       0   
35587           [

In [248]:
df.shape

(45466, 24)

In [249]:
# Dropping NA values
df["id"] =pd.to_numeric(df['id'], errors='coerce',downcast="integer")
# df["imdb_id"] =pd.to_numeric(df['imdb_id'], errors='coerce',downcast="integer")
df.dropna(subset=["id"],inplace=True)
df.shape

(45463, 24)

In [250]:
df.dropna(subset=["imdb_id"]).shape

(45446, 24)

In [251]:
# print(df.shape)
print("df_credits",df_credits.shape)

df_credits (45476, 3)


In [252]:
df_credits.head(3)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602


In [253]:
df.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862.0,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844.0,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602.0,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [254]:
df = df.merge(df_credits,on=["id"],how="left")

In [255]:
print(df_links.shape)
df_links.head()


(45843, 3)


,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [256]:
df = df.merge(df_keywords,on=["id"],how="left")

In [257]:
print("missing imdb ids in main data:",df["imdb_id"].isnull().sum())
print("LINK data missing Imdb ids:",df_links["imdbId"].isnull().sum())
print("LINK data missing Tmdb ids:",df_links["tmdbId"].isnull().sum())
# looks like our coercion lost many rows. We don't really care about imdb IDs so we can ignore frankly..

missing imdb ids in main data: 17
LINK data missing Imdb ids: 0
LINK data missing Tmdb ids: 219


In [258]:
# we see matching is poor for ID-movieId- but do we care? 
df.drop(["imdb_id"],axis=1).merge(df_links,left_on="id",right_on="movieId",how="inner").shape

(7767, 29)

In [259]:
# imdbID match rate: 
df.merge(df_links,left_on="imdb_id",right_on="imdbId",how="inner").shape

(0, 30)

In [260]:
df.shape

(46629, 27)

In [261]:
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [262]:
df.shape

(46629, 27)

In [263]:
df.drop_duplicates().shape

(45463, 27)

# some data cleaning:
- replace 0 budget with NaNs..
- drop row with missing cast/crew

In [264]:
df.isnull().sum()

adult                        0
belongs_to_collection    42055
budget                       0
genres                       0
homepage                 38620
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   995
popularity                   4
poster_path                399
production_companies         4
production_countries         4
release_date                88
revenue                      4
runtime                    268
spoken_languages             4
status                      86
tagline                  25846
title                        4
video                        4
vote_average                 4
vote_count                   4
cast                         1
crew                         1
keywords                     1
dtype: int64

In [265]:
df.dropna(subset=["cast","crew","keywords","popularity"],inplace=True)
print(df.shape)

(46624, 27)


In [266]:
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
df['budget'] = df['budget'].replace(0, np.nan)

df['revenue'] = df['revenue'].replace(0, np.nan)

In [267]:
vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()

m = vote_counts.quantile(0.75)

def weighted_rating(x):
    v = x['vote_count']+1 # added +1 - Dan
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

df['weighted_rating'] = df.apply(weighted_rating, axis=1)

In [268]:
df['genres'] = df['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [269]:
df['cast'] = df['cast'].apply(literal_eval)
df['crew'] = df['crew'].apply(literal_eval)
df['keywords'] = df['keywords'].apply(literal_eval)
df['cast_size'] = df['cast'].apply(lambda x: len(x))
df['crew_size'] = df['crew'].apply(lambda x: len(x))

In [270]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan
df['director'] = df['crew'].apply(get_director)

In [271]:

df['cast'] = df['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
df['cast'] = df['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

df['keywords'] = df['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [272]:
df['production_countries'].head()

0    [{'iso_3166_1': 'US', 'name': 'United States o...
1    [{'iso_3166_1': 'US', 'name': 'United States o...
2    [{'iso_3166_1': 'US', 'name': 'United States o...
3    [{'iso_3166_1': 'US', 'name': 'United States o...
4    [{'iso_3166_1': 'US', 'name': 'United States o...
Name: production_countries, dtype: object

In [273]:
df['production_countries'] = df['production_countries'].fillna('[]').apply(ast.literal_eval)
df['production_countries'] = df['production_countries'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])


In [274]:
df['production_countries'].head(2)

0    [United States of America]
1    [United States of America]
Name: production_countries, dtype: object

In [275]:
df['revenue_divide_budget'] = df['revenue'] / df['budget']


In [276]:
df['belongs_to_collection'].head()

0    {'id': 10194, 'name': 'Toy Story Collection', ...
1                                                  NaN
2    {'id': 119050, 'name': 'Grumpy Old Men Collect...
3                                                  NaN
4    {'id': 96871, 'name': 'Father of the Bride Col...
Name: belongs_to_collection, dtype: object

In [277]:
df['belongs_to_collection'] = df['belongs_to_collection'].fillna("[]").apply(ast.literal_eval).apply(lambda x: x['name'] if isinstance(x, dict) else np.nan)

In [278]:
## https://www.kaggle.com/rounakbanik/the-story-of-film
df['spoken_languages'] = df['spoken_languages'].fillna('[]').apply(ast.literal_eval).apply(lambda x: len(x) if isinstance(x, list) else np.nan)


In [279]:
df.head(20)['production_companies'].fillna("[]").apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

0     []
1     []
2     []
3     []
4     []
5     []
6     []
7     []
8     []
9     []
10    []
11    []
12    []
13    []
14    []
15    []
16    []
17    []
18    []
19    []
Name: production_companies, dtype: object

In [280]:
df['production_companies'] = df['production_companies'].apply(ast.literal_eval)
df['production_companies'] = df['production_companies'].fillna("[]").apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [281]:
df

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,vote_average,vote_count,cast,crew,keywords,weighted_rating,cast_size,crew_size,director,revenue_divide_budget
0,False,Toy Story Collection,30000000.0,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,7.7,5415.0,"[Tom Hanks, Tim Allen, Don Rickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousy, toy, boy, friendship, friends, riva...",7.684645,13,106,John Lasseter,12.451801
1,False,NaN,65000000.0,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,6.9,2413.0,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[board game, disappearance, based on children'...",6.876926,26,16,Joe Johnston,4.043035
2,False,Grumpy Old Men Collection,NaN,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,6.5,92.0,"[Walter Matthau, Jack Lemmon, Ann-Margret]","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[fishing, best friend, duringcreditsstinger, o...",6.162328,7,4,Howard Deutch,NaN
3,False,NaN,16000000.0,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,6.1,34.0,"[Whitney Houston, Angela Bassett, Loretta Devine]","[{'credit_id': '52fe44779251416c91011acb', 'de...","[based on novel, interracial relationship, sin...",5.675590,10,10,Forest Whitaker,5.090760
4,False,Father of the Bride Collection,NaN,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,5.7,173.0,"[Steve Martin, Diane Keaton, Martin Short]","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[baby, midlife crisis, confidence, aging, daug...",5.624595,12,7,Charles Shyer,NaN
5,False,NaN,60000000.0,"[Action, Crime, Drama, Thriller]",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,7.7,1886.0,"[Al Pacino, Robert De Niro, Val Kilmer]","[{'credit_id': '52fe4292c3a36847f802916d', 'de...","[robbery, detective, bank, obsession, chase, s...",7.656437,65,71,Michael Mann,3.123947
6,False,NaN,58000000.0,"[Comedy, Romance]",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,...,6.2,141.0,"[Harrison Ford, Julia Ormond, Greg Kinnear]","[{'credit_id': '52fe44959251416c75039da9', 'de...","[paris, brother brother relationship, chauffeu...",6.014294,57,53,Sydney Pollack,NaN
7,False,NaN,NaN,"[Action, Adventure, Drama, Family]",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",...,5.4,45.0,"[Jonathan Taylor Thomas, Brad Renfro, Rachael ...","[{'credit_id': '52fe46bdc3a36847f810f797', 'de...",[],5.331446,7,4,Peter Hewitt,NaN
8,False,NaN,35000000.0,"[Action, Adventure, Thriller]",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,5.5,174.0,"[Jean-Claude Van Damme, Powers Boothe, Dorian ...","[{'credit_id': '52fe44dbc3a36847f80ae0f1', 'de...","[terrorist, hostage, explosive, vice president]",5.457491,6,9,Peter Hyams,1.838576
9,False,James Bond Collection,58000000.0,"[Adventure, Action, Thriller]",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,6.6,1194.0,"[Pierce Brosnan, Sean Bean, Izabella Scorupco]","[{'credit_id': '52fe426ec3a36847f801e14b', 'de...","[cuba, falsely accused, secret identity, compu...",6.562340,20,46,Martin Campbell,6.072311


In [282]:
df.to_csv("movies_Merged_Alldata.csv.gz",index=False,compression="gzip")

# Converting to Dummies 

genres, director, original_language, production_companies, production_countries, cast, crew, keywords

In [283]:
from collections import Counter

# Find top 50 actors who appear most frequently
counter = Counter([item for sublist in df['cast'].values for item in sublist])
top_actors = counter.most_common(50)

In [284]:
# For each movie, check whether a top-50 actor appeared
for name, count in top_actors:
    print(name)
    
    for index, row in df.iterrows():
        list_of_names = row['cast']
    
        if name in list_of_names:
            df['cast_'+name] = 1
        else:
            df['cast_'+name] = 0

John Wayne
Jackie Chan
Robert De Niro
Michael Caine
Gérard Depardieu
Christopher Lee
Nicolas Cage
Harvey Keitel
Jeff Bridges
James Mason
Bette Davis
Samuel L. Jackson
Susan Sarandon
Donald Sutherland
Burt Lancaster
Bruce Willis
Burt Reynolds
Gene Hackman
Cary Grant
Marcello Mastroianni
James Stewart
Henry Fonda
Anthony Quinn
Eric Roberts
Gary Cooper
Barbara Stanwyck
Kirk Douglas
Robert Mitchum
Morgan Freeman
Catherine Deneuve
Helen Mirren
Paul Newman
Alec Baldwin
Christopher Walken
Boris Karloff
Vincent Price
Dolph Lundgren
Antonio Banderas
John Cusack
Meryl Streep
James Caan
Joan Crawford
James Franco
Akshay Kumar
Tom Hanks
Robin Williams
Val Kilmer
Julianne Moore
Isabelle Huppert
Robert Duvall


In [285]:
df

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,cast_James Caan,cast_Joan Crawford,cast_James Franco,cast_Akshay Kumar,cast_Tom Hanks,cast_Robin Williams,cast_Val Kilmer,cast_Julianne Moore,cast_Isabelle Huppert,cast_Robert Duvall
0,False,Toy Story Collection,30000000.0,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,0,0,0,0,0,0,0,0,0,0
1,False,NaN,65000000.0,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,0,0,0,0,0,0,0,0,0,0
2,False,Grumpy Old Men Collection,NaN,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0,0,0,0,0,0,0,0,0,0
3,False,NaN,16000000.0,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,0,0,0,0,0,0,0,0,0,0
4,False,Father of the Bride Collection,NaN,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,0,0,0,0,0,0,0,0,0,0
5,False,NaN,60000000.0,"[Action, Crime, Drama, Thriller]",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,0,0,0,0,0,0,0,0,0,0
6,False,NaN,58000000.0,"[Comedy, Romance]",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,...,0,0,0,0,0,0,0,0,0,0
7,False,NaN,NaN,"[Action, Adventure, Drama, Family]",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",...,0,0,0,0,0,0,0,0,0,0
8,False,NaN,35000000.0,"[Action, Adventure, Thriller]",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,0,0,0,0,0,0,0,0,0,0
9,False,James Bond Collection,58000000.0,"[Adventure, Action, Thriller]",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,0,0,0,0,0,0,0,0,0,0


In [286]:
# Find top 50 genres which are most poplular
counter = Counter([item for sublist in df['genres'].values for item in sublist])
top_genres = counter.most_common(50)

In [287]:
# For each movie, check whether a top-50 genres
for name, count in top_genres:
    print(name)
    
    for index, row in df.iterrows():
        list_of_names = row['genres']
    
        if name in list_of_names:
            df['genres'+name] = 1
        else:
            df['genres'+name] = 0

Drama
Comedy
Thriller
Romance
Action
Horror
Crime
Documentary
Adventure
Science Fiction
Family
Mystery
Fantasy
Animation
Foreign
Music
History
War
Western
TV Movie


In [288]:
df

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,genresFamily,genresMystery,genresFantasy,genresAnimation,genresForeign,genresMusic,genresHistory,genresWar,genresWestern,genresTV Movie
0,False,Toy Story Collection,30000000.0,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,0,0,0,0,0,0,0,0,0,0
1,False,NaN,65000000.0,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,0,0,0,0,0,0,0,0,0,0
2,False,Grumpy Old Men Collection,NaN,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0,0,0,0,0,0,0,0,0,0
3,False,NaN,16000000.0,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,0,0,0,0,0,0,0,0,0,0
4,False,Father of the Bride Collection,NaN,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,0,0,0,0,0,0,0,0,0,0
5,False,NaN,60000000.0,"[Action, Crime, Drama, Thriller]",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,0,0,0,0,0,0,0,0,0,0
6,False,NaN,58000000.0,"[Comedy, Romance]",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,...,0,0,0,0,0,0,0,0,0,0
7,False,NaN,NaN,"[Action, Adventure, Drama, Family]",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",...,0,0,0,0,0,0,0,0,0,0
8,False,NaN,35000000.0,"[Action, Adventure, Thriller]",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,0,0,0,0,0,0,0,0,0,0
9,False,James Bond Collection,58000000.0,"[Adventure, Action, Thriller]",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,0,0,0,0,0,0,0,0,0,0


In [289]:
# Find top 50 keywords which are most poplular
counter = Counter([item for sublist in df['keywords'].values for item in sublist])
top_keywords = counter.most_common(50)

In [290]:
# For each keyword, check whether a top-50 keywords
for name, count in top_keywords:
    print(name)
    
    for index, row in df.iterrows():
        list_of_names = row['keywords']
    
        if name in list_of_names:
            df['keywords'+name] = 1
        else:
            df['keywords'+name] = 0

woman director
independent film
murder
based on novel
musical
sex
violence
biography
nudity
revenge
suspense
love
female nudity
sport
police
sequel
teenager
duringcreditsstinger
friendship
world war ii
drug
prison
stand-up comedy
high school
martial arts
suicide
film noir
rape
kidnapping
silent film
family
serial killer
monster
alien
paris
dystopia
new york
gay
blood
marriage
short
christmas
gore
death
zombie
gangster
small town
london england
romance
male nudity


In [291]:
df

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,keywordsshort,keywordschristmas,keywordsgore,keywordsdeath,keywordszombie,keywordsgangster,keywordssmall town,keywordslondon england,keywordsromance,keywordsmale nudity
0,False,Toy Story Collection,30000000.0,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,0,0,0,0,0,0,0,0,0,0
1,False,NaN,65000000.0,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,0,0,0,0,0,0,0,0,0,0
2,False,Grumpy Old Men Collection,NaN,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0,0,0,0,0,0,0,0,0,0
3,False,NaN,16000000.0,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,0,0,0,0,0,0,0,0,0,0
4,False,Father of the Bride Collection,NaN,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,0,0,0,0,0,0,0,0,0,0
5,False,NaN,60000000.0,"[Action, Crime, Drama, Thriller]",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,0,0,0,0,0,0,0,0,0,0
6,False,NaN,58000000.0,"[Comedy, Romance]",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,...,0,0,0,0,0,0,0,0,0,0
7,False,NaN,NaN,"[Action, Adventure, Drama, Family]",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",...,0,0,0,0,0,0,0,0,0,0
8,False,NaN,35000000.0,"[Action, Adventure, Thriller]",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,0,0,0,0,0,0,0,0,0,0
9,False,James Bond Collection,58000000.0,"[Adventure, Action, Thriller]",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,0,0,0,0,0,0,0,0,0,0


In [292]:
df = pd.get_dummies(df, columns=['original_language'])
df

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_title,overview,popularity,...,original_language_tl,original_language_tr,original_language_uk,original_language_ur,original_language_uz,original_language_vi,original_language_wo,original_language_xx,original_language_zh,original_language_zu
0,False,Toy Story Collection,30000000.0,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,...,0,0,0,0,0,0,0,0,0,0
1,False,NaN,65000000.0,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,...,0,0,0,0,0,0,0,0,0,0
2,False,Grumpy Old Men Collection,NaN,"[Romance, Comedy]",NaN,15602,tt0113228,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,...,0,0,0,0,0,0,0,0,0,0
3,False,NaN,16000000.0,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,...,0,0,0,0,0,0,0,0,0,0
4,False,Father of the Bride Collection,NaN,[Comedy],NaN,11862,tt0113041,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,...,0,0,0,0,0,0,0,0,0,0
5,False,NaN,60000000.0,"[Action, Crime, Drama, Thriller]",NaN,949,tt0113277,Heat,"Obsessive master thief, Neil McCauley leads a ...",17.9249,...,0,0,0,0,0,0,0,0,0,0
6,False,NaN,58000000.0,"[Comedy, Romance]",NaN,11860,tt0114319,Sabrina,An ugly duckling having undergone a remarkable...,6.67728,...,0,0,0,0,0,0,0,0,0,0
7,False,NaN,NaN,"[Action, Adventure, Drama, Family]",NaN,45325,tt0112302,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",2.56116,...,0,0,0,0,0,0,0,0,0,0
8,False,NaN,35000000.0,"[Action, Adventure, Thriller]",NaN,9091,tt0114576,Sudden Death,International action superstar Jean Claude Van...,5.23158,...,0,0,0,0,0,0,0,0,0,0
9,False,James Bond Collection,58000000.0,"[Adventure, Action, Thriller]",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,GoldenEye,James Bond must unmask the mysterious head of ...,14.686,...,0,0,0,0,0,0,0,0,0,0


In [293]:
df = pd.get_dummies(df,columns=['director'])
df

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_title,overview,popularity,...,director_Геннадий Сокольский,director_Григорий Никулин,director_Михаил Калатозишвили,director_Михаил Пореченков,director_Федор Дмитриев,director_Ярополк Лапшин,director_پیمان معادی,director_塩谷 直義,director_杰森·莫玛,director_진모영
0,False,Toy Story Collection,30000000.0,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,...,0,0,0,0,0,0,0,0,0,0
1,False,NaN,65000000.0,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,...,0,0,0,0,0,0,0,0,0,0
2,False,Grumpy Old Men Collection,NaN,"[Romance, Comedy]",NaN,15602,tt0113228,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,...,0,0,0,0,0,0,0,0,0,0
3,False,NaN,16000000.0,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,...,0,0,0,0,0,0,0,0,0,0
4,False,Father of the Bride Collection,NaN,[Comedy],NaN,11862,tt0113041,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,...,0,0,0,0,0,0,0,0,0,0
5,False,NaN,60000000.0,"[Action, Crime, Drama, Thriller]",NaN,949,tt0113277,Heat,"Obsessive master thief, Neil McCauley leads a ...",17.9249,...,0,0,0,0,0,0,0,0,0,0
6,False,NaN,58000000.0,"[Comedy, Romance]",NaN,11860,tt0114319,Sabrina,An ugly duckling having undergone a remarkable...,6.67728,...,0,0,0,0,0,0,0,0,0,0
7,False,NaN,NaN,"[Action, Adventure, Drama, Family]",NaN,45325,tt0112302,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",2.56116,...,0,0,0,0,0,0,0,0,0,0
8,False,NaN,35000000.0,"[Action, Adventure, Thriller]",NaN,9091,tt0114576,Sudden Death,International action superstar Jean Claude Van...,5.23158,...,0,0,0,0,0,0,0,0,0,0
9,False,James Bond Collection,58000000.0,"[Adventure, Action, Thriller]",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,GoldenEye,James Bond must unmask the mysterious head of ...,14.686,...,0,0,0,0,0,0,0,0,0,0


In [296]:
# Find top 50 genres which are most poplular
counter = Counter([item for sublist in df['production_countries'].values for item in sublist])
top_production_countries = counter.most_common(50)

In [297]:
# For each movie, check whether a top-50 actor appeared
for name, count in top_production_countries:
    print(name)
    
    for index, row in df.iterrows():
        list_of_names = row['genres']
    
        if name in list_of_names:
            df['production_countries'+name] = 1
        else:
            df['production_countries'+name] = 0

United States of America
United Kingdom
France
Germany
Italy
Canada
Japan
Spain
Russia
India
Sweden
Hong Kong
Australia
South Korea
Belgium
Finland
Denmark
China
Netherlands
Mexico
Poland
Brazil
Argentina
Switzerland
Ireland
Norway
Austria
Czech Republic
Hungary
Greece
Turkey
Israel
Romania
Portugal
Taiwan
South Africa
Thailand
New Zealand
Iran
Serbia
Luxembourg
Philippines
Croatia
Chile
Iceland
Estonia
Bulgaria
Ukraine
Singapore
Colombia


In [298]:
df

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_title,overview,popularity,...,production_countriesLuxembourg,production_countriesPhilippines,production_countriesCroatia,production_countriesChile,production_countriesIceland,production_countriesEstonia,production_countriesBulgaria,production_countriesUkraine,production_countriesSingapore,production_countriesColombia
0,False,Toy Story Collection,30000000.0,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,...,0,0,0,0,0,0,0,0,0,0
1,False,NaN,65000000.0,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,...,0,0,0,0,0,0,0,0,0,0
2,False,Grumpy Old Men Collection,NaN,"[Romance, Comedy]",NaN,15602,tt0113228,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,...,0,0,0,0,0,0,0,0,0,0
3,False,NaN,16000000.0,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,...,0,0,0,0,0,0,0,0,0,0
4,False,Father of the Bride Collection,NaN,[Comedy],NaN,11862,tt0113041,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,...,0,0,0,0,0,0,0,0,0,0
5,False,NaN,60000000.0,"[Action, Crime, Drama, Thriller]",NaN,949,tt0113277,Heat,"Obsessive master thief, Neil McCauley leads a ...",17.9249,...,0,0,0,0,0,0,0,0,0,0
6,False,NaN,58000000.0,"[Comedy, Romance]",NaN,11860,tt0114319,Sabrina,An ugly duckling having undergone a remarkable...,6.67728,...,0,0,0,0,0,0,0,0,0,0
7,False,NaN,NaN,"[Action, Adventure, Drama, Family]",NaN,45325,tt0112302,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",2.56116,...,0,0,0,0,0,0,0,0,0,0
8,False,NaN,35000000.0,"[Action, Adventure, Thriller]",NaN,9091,tt0114576,Sudden Death,International action superstar Jean Claude Van...,5.23158,...,0,0,0,0,0,0,0,0,0,0
9,False,James Bond Collection,58000000.0,"[Adventure, Action, Thriller]",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,GoldenEye,James Bond must unmask the mysterious head of ...,14.686,...,0,0,0,0,0,0,0,0,0,0


In [299]:
# Find top 50 production companies who appear most frequently
counter = Counter([item for sublist in df['production_companies'].values for item in sublist])
top_production_companies = counter.most_common(50)

In [300]:
# For each movie, check whether a top-50 actor appeared
for name, count in top_production_companies:
    print(name)
    
    for index, row in df.iterrows():
        list_of_names = row['genres']
    
        if name in list_of_names:
            df['production_companies'+name] = 1
        else:
            df['production_companies'+name] = 0

Warner Bros.
Metro-Goldwyn-Mayer (MGM)
Paramount Pictures
Twentieth Century Fox Film Corporation
Universal Pictures
Canal+
Columbia Pictures Corporation
Columbia Pictures
RKO Radio Pictures
United Artists
New Line Cinema
Walt Disney Pictures
Touchstone Pictures
TriStar Pictures
Mosfilm
Miramax Films
France 2 Cinéma
Centre National de la Cinématographie (CNC)
Toho Company
BBC Films
Gaumont
StudioCanal
British Broadcasting Corporation (BBC)
Relativity Media
TF1 Films Production
Walt Disney Productions
Orion Pictures
Village Roadshow Pictures
Regency Enterprises
Lionsgate
Universal International Pictures (UI)
Zweites Deutsches Fernsehen (ZDF)
Dimension Films
DreamWorks SKG
Hammer Film Productions
Wild Bunch
Fox Searchlight Pictures
Rai Cinema
American International Pictures (AIP)
Film i Väst
Working Title Films
Svensk Filmindustri (SF)
Nordisk Film
Hollywood Pictures
Channel Four Films
Shaw Brothers
CJ Entertainment
Lions Gate Films
The Rank Organisation
Film4


In [301]:
df

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_title,overview,popularity,...,production_companiesWorking Title Films,production_companiesSvensk Filmindustri (SF),production_companiesNordisk Film,production_companiesHollywood Pictures,production_companiesChannel Four Films,production_companiesShaw Brothers,production_companiesCJ Entertainment,production_companiesLions Gate Films,production_companiesThe Rank Organisation,production_companiesFilm4
0,False,Toy Story Collection,30000000.0,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,...,0,0,0,0,0,0,0,0,0,0
1,False,NaN,65000000.0,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,...,0,0,0,0,0,0,0,0,0,0
2,False,Grumpy Old Men Collection,NaN,"[Romance, Comedy]",NaN,15602,tt0113228,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,...,0,0,0,0,0,0,0,0,0,0
3,False,NaN,16000000.0,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,...,0,0,0,0,0,0,0,0,0,0
4,False,Father of the Bride Collection,NaN,[Comedy],NaN,11862,tt0113041,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,...,0,0,0,0,0,0,0,0,0,0
5,False,NaN,60000000.0,"[Action, Crime, Drama, Thriller]",NaN,949,tt0113277,Heat,"Obsessive master thief, Neil McCauley leads a ...",17.9249,...,0,0,0,0,0,0,0,0,0,0
6,False,NaN,58000000.0,"[Comedy, Romance]",NaN,11860,tt0114319,Sabrina,An ugly duckling having undergone a remarkable...,6.67728,...,0,0,0,0,0,0,0,0,0,0
7,False,NaN,NaN,"[Action, Adventure, Drama, Family]",NaN,45325,tt0112302,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",2.56116,...,0,0,0,0,0,0,0,0,0,0
8,False,NaN,35000000.0,"[Action, Adventure, Thriller]",NaN,9091,tt0114576,Sudden Death,International action superstar Jean Claude Van...,5.23158,...,0,0,0,0,0,0,0,0,0,0
9,False,James Bond Collection,58000000.0,"[Adventure, Action, Thriller]",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,GoldenEye,James Bond must unmask the mysterious head of ...,14.686,...,0,0,0,0,0,0,0,0,0,0


In [302]:
df.shape

(46624, 17910)

In [ ]:
df.to_csv("Data/movies_Finaldata.csv.gz",index=False,compression="gzip")